# Base de datos Bsale


Carga de librerias que seran utilizadas

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import re
import warnings
from IPython.display import display

warnings.filterwarnings("ignore")

DISPLAY_OPTIONS = {
    "display.max_rows": None,
    "display.max_columns": None,
    "display.width": None,
    "display.max_colwidth": None
}
for option, value in DISPLAY_OPTIONS.items():
    pd.set_option(option, value)

### Ruta archivos

In [3]:
BASE_DIR = Path(r"C:\Nueva carpeta")

ARCHIVOS = {
    "ventas_productos": "venta por producto oct 25.xlsx",
    "ventas_html": "ventas oct25.xls",
    "costos": "Base SKU.xlsx"
}

def _resolver(nombre: str) -> Path:
    ruta = BASE_DIR / nombre
    if not ruta.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {ruta}")
    return ruta

def leer_excel(alias: str, **kwargs):
    return pd.read_excel(_resolver(ARCHIVOS[alias]), **kwargs)

def leer_html(alias: str, **kwargs):
    return pd.read_html(_resolver(ARCHIVOS[alias]), **kwargs)

### Carga funciones

In [4]:
# Funciones

def autoformatear_hoja(writer, sheet_name, df, percent_cols=None):
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    header_fmt = workbook.add_format({
        "bold": True,
        "bg_color": "#0F4C81",
        "font_color": "#FFFFFF",
        "align": "center",
        "valign": "vcenter",
        "border": 1
    })
    num_fmt = workbook.add_format({"num_format": "#,##0", "align": "right"})
    pct_fmt = workbook.add_format({"num_format": '0.00"%"', "align": "right"})
    text_fmt = workbook.add_format({"align": "left"})
    worksheet.freeze_panes(1, 1)
    worksheet.set_row(0, 20)
    percent_cols = percent_cols or []
    for col_idx, col in enumerate(df.columns):
        worksheet.write(0, col_idx, col, header_fmt)
        serie = df[col].astype(str).replace("nan", "")
        width = min(max([len(col)] + serie.str.len().tolist()) + 2, 40)
        if col in percent_cols:
            fmt = pct_fmt
        elif pd.api.types.is_numeric_dtype(df[col]):
            fmt = num_fmt
        else:
            fmt = text_fmt
        worksheet.set_column(col_idx, col_idx, width, fmt)

def normalizar_columnas(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df

def mostrar_resumen(nombre, df, n=3):
    filas, columnas = df.shape
    print(f"{nombre}: {filas} filas x {columnas} columnas")
    display(df.head(n))
    print(df.dtypes.head())

def agregar_costo_unitario_8dig(Base_ventas, Costos):
    # Copias para no modificar los originales
    base = Base_ventas.copy()
    costos = Costos.copy()

    # --- LIMPIAR Costos ---
    costos['ARTICULO'] = costos['ARTICULO'].astype(str).str.strip()

    # Quitar filas basura (sin código o con "Total")
    mask_valid = (
        costos['ARTICULO'].notna()
        & (costos['ARTICULO'] != '')
        & (costos['ARTICULO'].str.lower() != 'total')
        & (costos['ARTICULO'].str.lower() != 'nan')
    )
    costos_limpio = costos[mask_valid].copy()

    # Clave en Costos (8 dígitos)
    costos_limpio['ARTICULO_key'] = costos_limpio['ARTICULO'].astype(str).str.strip().str[:8]

    # Nos quedamos solo con clave + costo unitario
    costos_limpio = costos_limpio[['ARTICULO_key', 'COSTO UNIT.']].drop_duplicates(
        subset='ARTICULO_key',
        keep='last'
    )

    # --- CLAVE EN BASE_VENTAS ---
    base['SKU_str'] = base['SKU'].astype(str).str.strip()
    base['SKU_key'] = base['SKU_str'].str[:8]   # solo primeros 8 dígitos

    # --- MERGE: Base_ventas a la izquierda ---
    base_merged = base.merge(
        costos_limpio,
        left_on='SKU_key',
        right_on='ARTICULO_key',
        how='left'
    )

    # Rellenar / crear la columna Costo neto unitario con COSTO UNIT.
    base_merged['Costo neto unitario'] = base_merged['COSTO UNIT.']

    # Limpiar columnas auxiliares
    base_merged = base_merged.drop(columns=['SKU_str', 'SKU_key', 'ARTICULO_key', 'COSTO UNIT.'])

    return base_merged


# Utilidades de normalización y lookup para marketplaces
def normalizar_tipo_y_numero(tipo, numero):
    tipo_norm = tipo.astype(str).str.strip().str.lower()
    num_norm = numero.astype(str).str.replace(r"\.0$", "", regex=True).str.strip()
    return tipo_norm + "||" + num_norm

def construir_lookup_marketplace(tables, col_tipo, col_num, col_mkt):
    filas = []
    for df in tables:
        if {col_tipo, col_num, col_mkt}.issubset(df.columns):
            tmp = df[[col_tipo, col_num, col_mkt]].copy()
            tmp["__key__"] = normalizar_tipo_y_numero(tmp[col_tipo], tmp[col_num])
            filas.append(tmp[["__key__", col_mkt]])
    if not filas:
        return pd.Series(dtype="object")
    lookup_df = (
        pd.concat(filas, ignore_index=True)
        .dropna(subset=["__key__"])
        .drop_duplicates(subset="__key__", keep="last")
    )
    return lookup_df.set_index("__key__")[col_mkt]

def construir_lookup_por_numero(tables, col_num, col_mkt):
    filas = []
    for df in tables:
        if {col_num, col_mkt}.issubset(df.columns):
            tmp = df[[col_num, col_mkt]].copy()
            tmp[col_num] = (
                tmp[col_num].astype(str).str.replace(r"\.0$", "", regex=True).str.strip()
            )
            filas.append(tmp.dropna(subset=[col_num]))
    if not filas:
        return pd.Series(dtype="object")
    lookup_df = (
        pd.concat(filas, ignore_index=True)
        .drop_duplicates(subset=col_num, keep="last")
        .set_index(col_num)[col_mkt]
    )
    return lookup_df

DOC_CLEAN_PATTERN = re.compile(r"[^0-9A-Za-z]")

def normalizar_numero_documento(valor):
    if pd.isna(valor):
        return ""
    cleaned = DOC_CLEAN_PATTERN.sub("", str(valor)).lstrip("0")
    return cleaned or "0"

def construir_lookup_por_doc_normalizado(tables, col_num, col_mkt):
    filas = []
    for df in tables:
        if {col_num, col_mkt}.issubset(df.columns):
            tmp = df[[col_num, col_mkt]].copy()
            tmp["__doc_clean__"] = tmp[col_num].apply(normalizar_numero_documento)
            filas.append(tmp.dropna(subset=["__doc_clean__"]))
    if not filas:
        return pd.Series(dtype="object")
    return (
        pd.concat(filas, ignore_index=True)
        .drop_duplicates(subset="__doc_clean__", keep="last")
        .set_index("__doc_clean__")[col_mkt]
    )

def agregar_marketplace_desde_tables(Base_ventas, tables):
    """
    Cruza Base_ventas con los marketplace encontrados en tables utilizando
    llaves normalizadas (tipo+número) y, si no hay tipo, sólo por número.
    """
    tablas_validas = [t for t in tables if {'tipo documento', 'nº documento', 'marketplace'}.issubset(t.columns)]
    lookup_directo = pd.Series(dtype="object")
    if tablas_validas:
        lookup_directo = construir_lookup_marketplace(
            tablas_validas,
            col_tipo='tipo documento',
            col_num='nº documento',
            col_mkt='marketplace'
        )

    lookup_por_num = construir_lookup_por_numero(tables, col_num='nº documento', col_mkt='marketplace')
    lookup_por_doc = construir_lookup_por_doc_normalizado(tables, col_num='nº documento', col_mkt='marketplace')

    Base_norm = Base_ventas.copy()
    if 'nº documento' not in Base_norm.columns:
        raise ValueError("Base_ventas debe incluir la columna 'nº documento'.")
    Base_norm['nº documento'] = (
        Base_norm['nº documento']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
        .str.strip()
    )
    Base_norm['__doc_clean__'] = Base_norm['nº documento'].apply(normalizar_numero_documento)

    tipo_candidates = [col for col in Base_norm.columns if col.strip().lower() == 'tipo documento']
    tiene_tipo = bool(tipo_candidates)
    if tiene_tipo and not lookup_directo.empty:
        col_tipo = tipo_candidates[0]
        Base_norm['__key__'] = normalizar_tipo_y_numero(Base_norm[col_tipo], Base_norm['nº documento'])
        Base_norm['marketplace'] = Base_norm['__key__'].map(lookup_directo)
    else:
        Base_norm['marketplace'] = np.nan

    if not lookup_por_num.empty:
        Base_norm['marketplace'] = Base_norm['marketplace'].fillna(
            Base_norm['nº documento'].map(lookup_por_num)
        )
    if not lookup_por_doc.empty:
        Base_norm['marketplace'] = Base_norm['marketplace'].fillna(
            Base_norm['__doc_clean__'].map(lookup_por_doc)
        )

    tablas_prov = [t for t in tables if {'tipo documento proveniente', 'nº documento proveniente', 'marketplace'}.issubset(t.columns)]
    if tablas_prov:
        if '__key__' in Base_norm.columns:
            lookup_proveniente = construir_lookup_marketplace(
                tablas_prov,
                col_tipo='tipo documento proveniente',
                col_num='nº documento proveniente',
                col_mkt='marketplace'
            )
            Base_norm['marketplace'] = Base_norm['marketplace'].fillna(
                Base_norm['__key__'].map(lookup_proveniente)
            )

        lookup_prov_num = construir_lookup_por_numero(
            tablas_prov,
            col_num='nº documento proveniente',
            col_mkt='marketplace'
        )
        lookup_prov_doc = construir_lookup_por_doc_normalizado(
            tablas_prov,
            col_num='nº documento proveniente',
            col_mkt='marketplace'
        )
        if not lookup_prov_num.empty:
            Base_norm['marketplace'] = Base_norm['marketplace'].fillna(
                Base_norm['nº documento'].map(lookup_prov_num)
            )
        if not lookup_prov_doc.empty:
            Base_norm['marketplace'] = Base_norm['marketplace'].fillna(
                Base_norm['__doc_clean__'].map(lookup_prov_doc)
            )
    if '__key__' in Base_norm.columns:
        Base_norm.drop(columns='__key__', inplace=True)
    if '__doc_clean__' in Base_norm.columns:
        Base_norm.drop(columns='__doc_clean__', inplace=True)
    Base_norm['marketplace'] = Base_norm['marketplace'].fillna('sin datos')
    return Base_norm

def construir_resumen(df, campo):
    resumen_tmp = (
        df.groupby(campo, dropna=False)
          .agg(agg_map)
          .reset_index()
    )
    if {'Venta Total Neta', 'Costo Total Neto'}.issubset(resumen_tmp.columns):
        resumen_tmp['Margen'] = resumen_tmp['Venta Total Neta'] - resumen_tmp['Costo Total Neto']
        resumen_tmp['% Margen'] = np.where(
            resumen_tmp['Venta Total Neta'] == 0,
            0,
            (resumen_tmp['Margen'] / resumen_tmp['Venta Total Neta']) * 100
        ).round(2)
    else:
        raise ValueError("Las columnas necesarias para calcular el margen no están presentes.")
    return resumen_tmp

def agregar_fila_total(df, campo):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    total = {campo: 'Total'}
    for col in numeric_cols:
        if col in {'Margen', '% Margen'}:
            continue
        total[col] = df[col].sum(min_count=1)
    total_df = pd.DataFrame([total])
    if {'Venta Total Neta', 'Costo Total Neto'}.issubset(total_df.columns):
        total_df['Margen'] = total_df['Venta Total Neta'] - total_df['Costo Total Neto']
        total_df['% Margen'] = np.where(
            total_df['Venta Total Neta'] == 0,
            0,
            (total_df['Margen'] / total_df['Venta Total Neta']) * 100
        ).round(2)
    return pd.concat([df, total_df], ignore_index=True)

# Limpieza de datos

In [5]:
# Normalización de datos de entrada
Base_ventas = normalizar_columnas(leer_excel("ventas_productos"))
Base_ventas.rename(columns={"Numero Documento": "nº documento"}, inplace=True)
Base = normalizar_columnas(leer_html("ventas_html", header=11)[0])
Costos = normalizar_columnas(leer_excel("costos"))


Base_ventas_con_costo = agregar_costo_unitario_8dig(Base_ventas, Costos)

In [6]:
# Eliminar ultima fila
Base_ventas = Base_ventas_con_costo[:-1]

# Conteo de productos para devoluciones columna 'Cantidad' (Venta Total Neta/Precio Neto Unitario = Cantidad )
Base_ventas['Cantidad'] = Base_ventas['Venta Total Neta'] / Base_ventas['Precio Neto Unitario']

mask_devol = Base_ventas['Tipo Movimiento'].str.lower() == 'devolucion'
Base_ventas.loc[mask_devol, 'Cantidad'] *= -1

# Para devoluciones, el Costo neto unitario debe ser negativo
Base_ventas.loc[mask_devol, 'Costo neto unitario'] *= -1

#Calculo Costo Total Neto para devolucion
Base_ventas.loc[mask_devol, 'Costo Total Neto'] = -Base_ventas.loc[mask_devol, 'Costo neto unitario'] * Base_ventas.loc[mask_devol, 'Cantidad']

# Recalculo Margen y %  Margen 
Base_ventas['Margen'] = Base_ventas['Venta Total Neta'] - Base_ventas['Costo Total Neto']
Base_ventas['% Margen'] = (Base_ventas['Margen'] / Base_ventas['Venta Total Neta']).round(2)*100

# Para devoluciones, el % Margen debe ser negativos
Base_ventas.loc[mask_devol, '% Margen'] *= -1


Base_ventas.head()

,Tipo Movimiento,Tipo de Documento,nº documento,Fecha de Emisión,Tracking number,Fecha y Hora Venta,Sucursal,Vendedor,Nombre Cliente,Cliente RUT,Email Cliente,Cliente Dirección,Cliente Comuna,Cliente Ciudad,Lista de Precio,Tipo de entrega,Moneda,Tipo de Producto / Servicio,SKU,Producto / Servicio,Variante,Otros Atributos,Marca,Detalle de Productos/Servicios Pack/Promo,Precio de Lista,Precio Neto Unitario,Precio Bruto Unitario,Cantidad,Venta Total Neta,Total Impuestos,Venta Total Bruta,Nombre de dcto,Descuento Neto,Descuento Bruto,% Descuento,Costo neto unitario,Costo Total Neto,Margen,% Margen
0,venta,BOLETA ELECTRÓNICA T,48273.0,01/10/2025,68dc9c3bbf662efb56f8cd98,01/10/2025 00:12:58,Casa Matriz,VENTA ONLINE,Vivian Porme,18017260-2,vivi.pormam@gmail.com,Santiago,Regi\u00f3n Metropolitana,Santiago,Lista de Precios Base,retiro en tienda,CLP,Zapatilla,1.254172e+13,Via Uno Zapatilla 12541718 Negro/Blanco,37,NaN,NaN,NaN,14990.0,12597.0,14990,1.0,12597.0,2393.0,14990.0,NaN,0.0,0.0,0.0,13472.0,13487.0,-890.0,-7.0
1,venta,BOLETA ELECTRÓNICA T,48274.0,01/10/2025,68dcae83bf662efb56f8d24b,01/10/2025 01:30:58,Casa Matriz,VENTA ONLINE,daniela ramirez,15786715-6,NaN,"avenida ernesto alvear 562, casa 562",Region Metropolitana,CORDILLERA,Lista de Precios Base,retiro en tienda,CLP,Sandalia,2.263362e+13,Via Uno Sandalia 22633620 Negro,40,NaN,NaN,NaN,32990.0,27723.0,32990,1.0,27723.0,5267.0,32990.0,NaN,0.0,0.0,0.0,8845.0,8848.0,18875.0,68.0
2,venta,BOLETA ELECTRÓNICA T,48275.0,01/10/2025,68dd15b44b24743bba4f01b0,01/10/2025 08:51:15,Casa Matriz,VENTA ONLINE,Caterina Sadler Barberis,18024280-5,caterina.sadler@gmail.com,Santiago,Regi\u00f3n Metropolitana,Santiago,Lista de Precios Base,retiro en tienda,CLP,Botin,1.253240e+13,Via Uno Botin 12532402 Negro,40,NaN,NaN,NaN,23196.0,19492.0,23196,1.0,19492.0,3704.0,23196.0,NaN,0.0,0.0,0.0,14460.0,14460.0,5032.0,26.0
3,venta,BOLETA ELECTRÓNICA T,48275.0,01/10/2025,68dd15b44b24743bba4f01b0,01/10/2025 08:51:15,Casa Matriz,VENTA ONLINE,Caterina Sadler Barberis,18024280-5,caterina.sadler@gmail.com,Santiago,Regi\u00f3n Metropolitana,Santiago,Lista de Precios Base,retiro en tienda,CLP,Sin Tipo,1.759319e+09,Glosa,Despacho,NaN,NaN,NaN,4786.0,4022.0,4786,1.0,4022.0,764.0,4786.0,NaN,0.0,0.0,0.0,NaN,0.0,4022.0,100.0
4,devolucion,NOTA DE CRÉDITO ELECTRÓNICA,8446.0,01/10/2025,NaN,01/10/2025 09:22:28,Casa Matriz,VENTA ONLINE,MARIBEL IRARRAZABAL MONJE,14202757-7,NaN,"Los canelos821, Casa",LIBERTADOR GENERAL BERNARDO O'HIGGINS,CACHAPOAL,Lista de Precios Base,retiro en tienda,CLP,Botin,1.253144e+13,Via Uno Botin 12531437 Negro,39,NaN,NaN,NaN,26990.0,-22681.0,26990,-1.0,-22681.0,-4309.0,-26990.0,NaN,0.0,0.0,0.0,-13801.0,-13801.0,-8880.0,-39.0


In [7]:
# Separacion de tablas en Base
Repeats_headers = ['Tipo Documento', 'Nº Documento', 'Rut cliente']

# Identificar las filas donde aparecen los encabezados
if 'Tipo Documento' not in Base.columns:
    raise KeyError("La hoja Base no contiene la columna 'Tipo Documento'.")
header_indices = Base[Base['Tipo Documento'].isin(Repeats_headers)].index.tolist()
if not header_indices:
    raise ValueError("No se encontraron encabezados repetidos en la columna 'Tipo Documento'.")

# Agregar la última fila como un límite para capturar la última tabla
header_indices.append(len(Base))

# Lista donde guardaremos las tablas separadas
tables = []

# Verificar si hay una tabla antes del primer encabezado
if header_indices[0] > 0:
    first_table = Base.iloc[:header_indices[0]].reset_index(drop=True)
    if not first_table.empty:
        tables.append(first_table)

# Separar las tablas detectadas y asignar los encabezados correctos
for i in range(len(header_indices) - 1):
    start = header_indices[i]  # Fila donde está el encabezado
    end = header_indices[i + 1]  # Hasta el siguiente encabezado

    # Extraer los datos y resetear el índice
    table = Base.iloc[start:end].reset_index(drop=True)
    
    # Asignar la primera fila como encabezado
    table.columns = table.iloc[0]  # Convertir la primera fila en nombres de columnas
    table = table[1:].reset_index(drop=True)  # Eliminar la fila de encabezado original
    if i < 4:
        table=table.iloc[:-2]

    if not table.empty:
        tables.append(table)

In [8]:
# Eliminar columnas vacías y filas completamente vacías en cada tabla
for i, table in enumerate(tables):
    table = table.dropna(axis=1, how='all')  # Eliminar columnas vacías
    tables[i] = table.reset_index(drop=True)  # Resetear el índice

In [9]:
# Normalizacion de nombres de columnas
Base.columns = Base.columns.str.strip().str.lower()
for i, table in enumerate(tables):
    table.columns = table.columns.str.strip().str.lower()
    tables[i] = table

In [10]:
tables[4].tail()

,tipo documento,nº documento,cliente,rut cliente,fecha emisión,fecha vencimiento,código tributario,fecha de generacion,fecha de declaracion,informado sii,monto neto documento,monto impuestos documento,monto exento documento,monto documento,emisor,vendedor,sucursal,estado,dirección,ciudad,comuna,documentos relacionados (tipo documento / nº documento / fecha emisión / monto del documento ),correo cliente
199,NOTA DE CRÉDITO ELECTRÓNICA,8481,Nathaly Malave,25701340-5,2025-10-06,2025-10-06,61,2025-10-06,2025-10-06,Informado,19311,3669,0,22980,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Av. El parque 1641,",Lampa,RM (Metropolitana),[BOLETA ELECTRÓNICA T/ 48182/ 2025-09-24/ 22980],NaN
200,NOTA DE CRÉDITO ELECTRÓNICA,8462,Julieta Herrera Godoy,19652894-6,2025-10-02,2025-10-02,61,2025-10-02,2025-10-02,Informado,17639,3351,0,20990,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Bulnes470, oficina 52",CHILLÁN,ÑUBLE,[BOLETA ELECTRÓNICA T/ 48139/ 2025-09-22/ 20990],NaN
201,NOTA DE CRÉDITO ELECTRÓNICA,8461,Carmen Gloria Pérez Echeverria,10588396-k,2025-10-02,2025-10-02,61,2025-10-02,2025-10-02,Informado,17639,3351,0,20990,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Gerardo Rebolledo 236,",Curacavi,RM (Metropolitana),[BOLETA ELECTRÓNICA T/ 48191/ 2025-09-25/ 20990],NaN
202,NOTA DE CRÉDITO ELECTRÓNICA,8449,Susana Olate Caceres,15635425-2,2025-10-01,2025-10-01,61,2025-10-01,2025-10-01,Informado,97461,18518,0,115979,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Sergio Reiss 18110, 56",COLINA,METROPOLITANA DE SANTIAGO,[BOLETA ELECTRÓNICA T/ 48206/ 2025-09-26/ 115980],NaN
203,NOTA DE CRÉDITO ELECTRÓNICA,8448,Tamara Nuez Cifuentes,17577463-7,2025-10-01,2025-10-01,61,2025-10-01,2025-10-01,Informado,39487,7503,0,46990,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Flamencos 2092,",SANTIAGO,REGION METROPOLITANA,[BOLETA ELECTRÓNICA T/ 48216/ 2025-09-27/ 46990],NaN


In [11]:
# n documento proveniente de columna documentos relacionados (tipo documento / nº documento / fecha emisión / monto del documento )
patron = r'\[(.*?)\s*/\s*([^/\]]+)\s*/'
col_rel = "documentos relacionados (tipo documento / nº documento / fecha emisión / monto del documento )"

for i, df in enumerate(tables):
    if col_rel in df.columns:
        extraidos = df[col_rel].astype(str).str.extract(patron)
        extraidos.columns = ["tipo documento proveniente", "nº documento proveniente"]
        tables[i][["tipo documento proveniente", "nº documento proveniente"]] = extraidos.apply(lambda c: c.str.strip())
tables[4].tail()

,tipo documento,nº documento,cliente,rut cliente,fecha emisión,fecha vencimiento,código tributario,fecha de generacion,fecha de declaracion,informado sii,monto neto documento,monto impuestos documento,monto exento documento,monto documento,emisor,vendedor,sucursal,estado,dirección,ciudad,comuna,documentos relacionados (tipo documento / nº documento / fecha emisión / monto del documento ),correo cliente,tipo documento proveniente,nº documento proveniente
199,NOTA DE CRÉDITO ELECTRÓNICA,8481,Nathaly Malave,25701340-5,2025-10-06,2025-10-06,61,2025-10-06,2025-10-06,Informado,19311,3669,0,22980,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Av. El parque 1641,",Lampa,RM (Metropolitana),[BOLETA ELECTRÓNICA T/ 48182/ 2025-09-24/ 22980],NaN,BOLETA ELECTRÓNICA T,48182
200,NOTA DE CRÉDITO ELECTRÓNICA,8462,Julieta Herrera Godoy,19652894-6,2025-10-02,2025-10-02,61,2025-10-02,2025-10-02,Informado,17639,3351,0,20990,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Bulnes470, oficina 52",CHILLÁN,ÑUBLE,[BOLETA ELECTRÓNICA T/ 48139/ 2025-09-22/ 20990],NaN,BOLETA ELECTRÓNICA T,48139
201,NOTA DE CRÉDITO ELECTRÓNICA,8461,Carmen Gloria Pérez Echeverria,10588396-k,2025-10-02,2025-10-02,61,2025-10-02,2025-10-02,Informado,17639,3351,0,20990,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Gerardo Rebolledo 236,",Curacavi,RM (Metropolitana),[BOLETA ELECTRÓNICA T/ 48191/ 2025-09-25/ 20990],NaN,BOLETA ELECTRÓNICA T,48191
202,NOTA DE CRÉDITO ELECTRÓNICA,8449,Susana Olate Caceres,15635425-2,2025-10-01,2025-10-01,61,2025-10-01,2025-10-01,Informado,97461,18518,0,115979,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Sergio Reiss 18110, 56",COLINA,METROPOLITANA DE SANTIAGO,[BOLETA ELECTRÓNICA T/ 48206/ 2025-09-26/ 115980],NaN,BOLETA ELECTRÓNICA T,48206
203,NOTA DE CRÉDITO ELECTRÓNICA,8448,Tamara Nuez Cifuentes,17577463-7,2025-10-01,2025-10-01,61,2025-10-01,2025-10-01,Informado,39487,7503,0,46990,San Mateo Spa ONLINE,VENTA ONLINE,Casa Matriz,Vigente,"Flamencos 2092,",SANTIAGO,REGION METROPOLITANA,[BOLETA ELECTRÓNICA T/ 48216/ 2025-09-27/ 46990],NaN,BOLETA ELECTRÓNICA T,48216


In [12]:
tables[3].tail()

,tipo documento,nº documento,cliente,rut cliente,fecha emisión,fecha vencimiento,código tributario,fecha de generacion,fecha de declaracion,informado sii,monto neto documento,monto impuestos documento,monto exento documento,monto documento,tipo de despacho,sucursal de destino,emisor,vendedor,sucursal,estado,dirección,ciudad,comuna
19,GUÍA DE DESPACHO ELECTRÓNICA,2315,FALABELLA RETAIL S.A.,77.261.280-K,2025-10-07,2025-10-07,52,2025-10-07,2025-10-07,Informado,0,0,0,0,Otros traslados no venta,No aplica,VENTA ONLINE,VENTA ONLINE,Casa Matriz,Vigente,PUENTE 530,SANTIAGO,SANTIAGO
20,GUÍA DE DESPACHO ELECTRÓNICA,2314,FALABELLA RETAIL S.A.,77.261.280-K,2025-10-06,2025-10-06,52,2025-10-06,2025-10-06,Informado,0,0,0,0,Otros traslados no venta,No aplica,VENTA ONLINE,VENTA ONLINE,Casa Matriz,Vigente,PUENTE 530,SANTIAGO,SANTIAGO
21,GUÍA DE DESPACHO ELECTRÓNICA,2313,FALABELLA RETAIL S.A.,77.261.280-K,2025-10-03,2025-10-03,52,2025-10-03,2025-10-03,Informado,0,0,0,0,Otros traslados no venta,No aplica,VENTA ONLINE,VENTA ONLINE,Casa Matriz,Vigente,PUENTE 530,SANTIAGO,SANTIAGO
22,GUÍA DE DESPACHO ELECTRÓNICA,2312,FALABELLA RETAIL S.A.,77.261.280-K,2025-10-02,2025-10-02,52,2025-10-02,2025-10-02,Informado,0,0,0,0,Otros traslados no venta,No aplica,VENTA ONLINE,VENTA ONLINE,Casa Matriz,Vigente,PUENTE 530,SANTIAGO,SANTIAGO
23,GUÍA DE DESPACHO ELECTRÓNICA,2311,FALABELLA RETAIL S.A.,77.261.280-K,2025-10-01,2025-10-01,52,2025-10-01,2025-10-01,Informado,0,0,0,0,Otros traslados no venta,No aplica,VENTA ONLINE,VENTA ONLINE,Casa Matriz,Vigente,PUENTE 530,SANTIAGO,SANTIAGO


# Cruce de datos

In [13]:


Base_final = agregar_marketplace_desde_tables(Base_ventas, tables)

# FACTURA DE EXPORTACIÓN ELECTRÓNICA siempre es marketplace cvd
Base_final.loc[
    Base_final['Tipo de Documento'] == 'FACTURA DE EXPORTACIÓN ELECTRÓNICA',
    'marketplace'
] = 'cvd'


num_cols_final = Base_final.select_dtypes(include=[np.number]).columns.tolist()
if '% Margen' in num_cols_final:
    num_cols_final.remove('% Margen')
for col in num_cols_final:
    Base_final[col] = (
        pd.to_numeric(Base_final[col], errors='coerce')
          .round(0)
          .astype('Int64')
    )
if '% Margen' in Base_final.columns:
    Base_final['% Margen'] = pd.to_numeric(Base_final['% Margen'], errors='coerce')

metricas = [
    'Precio de Lista',
    'Precio Neto Unitario',
    'Precio Bruto Unitario',
    'Cantidad',
    'Venta Total Neta',
    'Total Impuestos',
    'Venta Total Bruta',
    'Costo neto unitario',
    'Costo Total Neto',
    'Margen',
    '% Margen'
]
metricas_presentes = [col for col in metricas if col in Base_final.columns]
if not metricas_presentes:
    raise ValueError("Base_final no contiene ninguna de las métricas solicitadas.")
for col in metricas_presentes:
    Base_final[col] = pd.to_numeric(Base_final[col], errors='coerce')
metricas_sum = [col for col in metricas_presentes if col not in ['% Margen', 'Margen']]
agg_map = {col: 'sum' for col in metricas_sum}

obligatorias = ['marketplace', 'Sucursal']
if not set(obligatorias).issubset(Base_final.columns):
    raise ValueError("Base_final debe contener las columnas 'marketplace' y 'Sucursal'.")



resumen_marketplace = agregar_fila_total(construir_resumen(Base_final, 'marketplace'), 'marketplace')
resumen_sucursal = agregar_fila_total(construir_resumen(Base_final, 'Sucursal'), 'Sucursal')
resumen_tipo_producto = agregar_fila_total(
    construir_resumen(Base_final, 'Tipo de Producto / Servicio'),
    'Tipo de Producto / Servicio'
)
percent_cols_base = ['% Margen'] if '% Margen' in Base_final.columns else []
percent_cols_resumen = ['% Margen'] if '% Margen' in resumen_marketplace.columns else []
percent_cols_tipo = ['% Margen'] if '% Margen' in resumen_tipo_producto.columns else []
output_path = r"C:\Nueva carpeta\Base_ventas_con_marketplace.xlsx"
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    Base_final.to_excel(writer, sheet_name="Base", index=False)
    resumen_marketplace.to_excel(writer, sheet_name="Resumen_Marketplace", index=False)
    resumen_sucursal.to_excel(writer, sheet_name="Resumen_Sucursal", index=False)
    resumen_tipo_producto.to_excel(writer, sheet_name="Resumen_TipoProducto", index=False)
    autoformatear_hoja(writer, "Base", Base_final, percent_cols_base)
    autoformatear_hoja(writer, "Resumen_Marketplace", resumen_marketplace, percent_cols_resumen)
    autoformatear_hoja(writer, "Resumen_Sucursal", resumen_sucursal, percent_cols_resumen)
    autoformatear_hoja(writer, "Resumen_TipoProducto", resumen_tipo_producto, percent_cols_tipo)
print(resumen_marketplace.head())
print(resumen_sucursal.head())
print(resumen_tipo_producto.head())

# Guardar
Base_final.to_excel("Base_ventas_con_marketplace.xlsx", index=False)


    marketplace  Precio de Lista  Precio Neto Unitario  Precio Bruto Unitario  \
0           cvd         34580553              34580553               34580553   
1          fcom         20510525              11562157               20510525   
2  mercadolibre          9948351               7012023                9948351   
3         paris          2145767               1092472                2145767   
4        ripley          1471950                870691                1471950   

   Cantidad  Venta Total Neta  Total Impuestos  Venta Total Bruta  \
0         1          34580553                0           34580553   
1       555          11562192          2196884           13759075   
2       321           7012031          1332320            8344351   
3        48           1092472           207575            1300047   
4        64            870696           165434            1036130   

   Costo neto unitario  Costo Total Neto    Margen  % Margen  
0                    0             